## GCP

In [ ]:
# %pip list | grep google-cloud-aiplatform
# %pip list | grep google-api-core
# %pip install google-cloud-aiplatform==1.43.0
# %pip install google-api-core==2.17.1

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, ChatSession, Part
import vertexai.preview.generative_models as generative_models

import geopandas as gpd
from shapely.geometry import Point

2024-05-14 05:19:14.420459: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-14 05:19:15.475877: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-05-14 05:19:15.475986: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [2]:
vertexai.init(project = "adsp-capstone-property-pilot", location = "us-central1")

## Model Setting - Gemini pro 1.0

In [3]:
model = GenerativeModel("gemini-1.0-pro")
chat = model.start_chat()

def get_chat_response(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        text_response.append(chunk.text)
    return "".join(text_response)

In [4]:
# Test chat
prompt = "Hello."
print(get_chat_response(chat, prompt))

Hello! 👋 I'm glad you're here. How can I help you today?


## Settings

In [5]:
import requests

url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

headers = {
	"X-RapidAPI-Key": "20d92ad976msh2cc2986e4b8788dp100dabjsn404bc4a80fe1",
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

## Import functions

In [6]:
import os
os.chdir('/home/jupyter/property_pilot/scripts')

import generate_api_filter
import fetch_property_info
import chat_response

In [7]:
%run /home/jupyter/property_pilot/scripts/chat_response.py

In [8]:
%run /home/jupyter/property_pilot/scripts/generate_api_filter.py

In [9]:
%run /home/jupyter/property_pilot/scripts/fetch_property_info.py

## Import prompts

In [10]:
os.chdir('/home/jupyter/property_pilot/prompts')

# prompt_classifier
file_path = 'instruction_classifier.txt'
try:
    with open(file_path, 'r') as file:
        instruction_prompt_classifier = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}") 
    
# prompt_apifilter
file_path = 'generate_prompt_apifilter.txt'
try:
    with open(file_path, 'r') as file:
        instruction_prompt_apifilter = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")
    
# prompt_property(for final output)
file_path = 'generate_prompt_property.txt'
try:
    with open(file_path, 'r') as file:
        instruction_prompt_property = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")


## Property Plot main function

In [13]:
def pp_chatbot(user_query):
    
    # User's quety classify
    prompt_classifier = generate_prompt_classifier(instruction_prompt_classifier, user_query)
    response_int = int(get_chat_response(chat, prompt_classifier))
    
    # if the user is looking for a property     
    if response_int == 1:

        # get prompt for api filter 
        prompt_apifilter = generate_prompt_apifilter(instruction_prompt_apifilter, user_query)

        # get api filter
        apifilter = get_chat_response(chat, prompt_apifilter)
        apifilter = extract_json_to_dict(apifilter)

        # get listings from zillow
        response = requests.get(url, headers = headers, params = apifilter)
        
        print(response)

        # get top 3 listings(sorted by newest)
        # Define fields to extract
        fields = ["propertyType", "address", "price", "bedrooms", "bathrooms", "detailUrl", "imgSrc", "longitude", "latitude"]
        top_properties = extract_properties(response, 3, fields)

        # get description of the properties
        top_properties = fetch_descriptions(top_properties)

        # get resoFacts(detail) of the properties
        keys_to_fetch_resoFacts = [
            'hasGarage', 'hasPetsAllowed', 'heating', 'cooling', 'flooring', 'appliances',
            'laundryFeatures', 'associationFee',
            'livingArea', 'taxAnnualAmount', 'parkingFeatures', 'stories'
        ]
        top_properties = fetch_resoFacts(top_properties, headers, keys_to_fetch_resoFacts)

        # get school info of the properties
        top_properties = fetch_schools(top_properties, headers)
        
        neighborhood_boundaries_path = '/home/jupyter/Kshitiz_Working/geo_export_825d7df4-a9cd-4cef-b3d7-2ec1adc30204.shp'
        neighborhood_info_path = '/home/jupyter/Kshitiz_Working/neighborhood_info_final.csv'
        
        neighborhoods = load_neighborhood_boundaries(neighborhood_boundaries_path)
        load_neighborhood_info(neighborhood_info_path)
        
        print(top_properties)
        
        top_properties = fetch_neighborhood(top_properties, neighborhoods)
        top_properties = fetch_neighborhood_info(top_properties, neighborhoods_info)
        
        # create property map
        property_map = create_property_map(top_properties)

        # change format
        fields_desc = fields + ['description', 'resoFacts', 'schools']
        property_info = format_properties(top_properties, fields_desc)

        # prompt for final output from Gemini
        prompt_final = generate_prompt_property(instruction_prompt_property, user_query, property_info, str(fields))

        # get final response from Gemini
        print(get_chat_response(chat, prompt_final))
        display(property_map)

    else:
        # print('The user is not looking for a property')
        # General reaponse from Gemini for now
        print(get_chat_response(chat, user_query))

## Use case 1
### property search - detail info

In [14]:
user_query = "I am looking for a two-bedroom apartment in Chicago."
pp_chatbot(user_query)

<Response [403]>
An error occurred: 'props'
[]


IndexError: list index out of range

In [ ]:
user_query = "Are these properties close to public transportation facilities?"
pp_chatbot(user_query)

## Use case 2
### Initial query: Neighborhood info

RAG

## Use case 3
### Initial query: Restaurant info

Yelp API